LOOK AT DATA

In [ ]:
SOURCE_DIR = ""
os.chdir(SOURCE_DIR")

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import os
import torch
import os
from antispoofing.src.data.util import BaseDataset, split_dataset
from torchvision import transforms
TRAIN_FRAC = 0.8
from biometrics.src.data.casia import CASIA, load_dataset, load_video

In [ ]:
mapl = {
    "1": [0, 0, 0, 1],  # 1 real
    "2": [0, 0, 0, 1],  # 1 real
    "3": [0, 0, 1, 0],  # 2 print attack
    "4": [0, 0, 1, 0],  # 2 print attack
    "5": [0, 1, 0, 0],  # 3 print mask attack
    "6": [0, 1, 0, 0],  # 3 print mask attack
    "7": [1, 0, 0, 0],  # 4 replay attack
    "8": [1, 0, 0, 0],   # 4 replay attack
    "HR_1": [0, 0, 0, 1],
    "HR_2": [0, 0, 1, 0],
    "HR_3": [0, 1, 0, 0],
    "HR_4": [1, 0, 0, 0]
}
IMG_SIZE=224
group = 1
video = 1
frames = load_video(f"storage/datasets/CASIA_faceAntisp/train_release/{group}/{video}.avi")
frames[0].shape
ft = torch.tensor(frames)
print(ft.shape)
plt.figure()
f, axarr = plt.subplots(2,2) 
axarr[0,0].imshow(frames[0], cmap='gray')
axarr[0,1].imshow(frames[30], cmap='gray')
axarr[1,0].imshow(frames[60], cmap='gray')
axarr[1,1].imshow(frames[90], cmap='gray')
len(frames[0])

In [ ]:

data = CASIA()
data.prepare_data()
data.setup()
print(data)

In [ ]:
training_dataset[100].shape
plt.imshow(training_dataset[100], 'gray')

In [ ]:
sample = training_dataset[100]
print(sample.shape)
#sample = sample.reshape(-1,28,28).swapaxes(1, 2)
#print(sample.shape)
#plt.imshow(r, 'gray')
type(sample)
import torch
from torchvision import transforms
from torch.nn.functional import normalize
sample = torch.from_numpy(sample)
print(sample)
#sample = sample.reshape() # reshape(-1, 28, 28).swapaxes(1, 2) tensorflow
#sample = normalize(sample, p=1.0, dim=0)
#type(sample)
sample = torch.reshape(sample,(-1,))
print("flatten: ",sample)
#sample = torch.swapaxes(sample, -1, 0)
#sample = sample.t()
print(sample.shape)
# import torchvision
# from torchvision import transforms
# train_data_path = "./train/"
# transforms = transforms.Compose([
#     transforms.Resize(64),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                     std=[0.229, 0.224, 0.225] )
#     ])
# train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=transforms)

In [ ]:
import pytorch_lightning as pl
from biometrics.src.lit_models.base import BaseLitModel
from biometrics.src.data.casia import CASIA
from biometrics.src.models.cnn import CNN

casia = CASIA()
model = CNN(casia.config())
lit_model_class = BaseLitModel
lit_model = lit_model_class(args=None, model=model)
logger = pl.loggers.TensorBoardLogger("training/logs")
early_stopping_callback = pl.callbacks.EarlyStopping(
    monitor="val_loss", mode="min", patience=10)
model_checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filename="{epoch:03d}-{val_loss:.3f}-{val_cer:.3f}", monitor="val_loss", mode="min"
)
callbacks = [early_stopping_callback, model_checkpoint_callback]
#trainer = pl.Trainer()
trainer = pl.Trainer(callbacks=callbacks, logger=logger, weights_save_path="training/logs")
trainer.tune(lit_model, datamodule=casia)
trainer.fit(lit_model, datamodule=casia)
# trainer.test(lit_model, datamodule=casia)